In [1]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [9]:
import gc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import norm, multivariate_normal
# import timedelta
from datetime import datetime, timedelta

In [10]:
quants = pd.read_csv('debug_logs/quant_next_step_recurrent_gaussian_qts_9000_1_9march.csv')
quants

,time_step,building,0.001,0.05,0.1,0.15,0.2,0.25,0.3,0.35,...,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.999
0,0,0,-1.901124,-1.663122,-0.970390,-0.799129,-0.300693,-0.042036,-0.004489,0.063345,...,0.576237,0.606327,0.929095,1.124846,1.332976,1.804060,1.993226,2.516597,3.485628,4.819335
1,0,1,-1.556944,-1.359842,-0.805858,-0.658256,-0.292483,-0.175907,-0.028039,-0.017737,...,0.440802,0.457149,0.678392,0.831638,1.022557,1.389201,1.605132,2.087495,2.832243,3.463335
2,0,2,-2.052951,-1.944761,-1.544397,-1.416605,-1.274623,-1.157707,-1.136541,-1.058848,...,-0.592189,-0.574804,-0.512410,-0.362804,-0.309317,-0.185912,-0.082179,0.077947,0.689624,1.866776
3,0,3,-1.935402,-1.722003,-1.280940,-1.060490,-0.663175,-0.480352,-0.380271,-0.264827,...,0.576425,0.896773,1.158166,1.322296,1.549180,1.891387,2.076135,2.451796,3.106730,3.396426
4,0,4,-1.872774,-1.701886,-1.281400,-1.095874,-0.824696,-0.652655,-0.652616,-0.611236,...,-0.194041,-0.111488,0.066191,0.182347,0.354344,0.545380,0.656373,0.854463,1.434433,2.504304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,8759,0,0.325116,1.327817,1.423375,1.555182,1.619344,1.684418,1.696235,1.734751,...,2.014136,2.054010,2.141009,2.208828,2.213158,2.443807,2.675630,3.123001,3.739150,5.468320
43796,8759,1,0.153467,0.939190,1.217416,1.394382,1.486564,1.682496,1.734810,1.839102,...,2.093065,2.162920,2.236235,2.327854,2.347003,2.417198,2.511297,2.805753,3.209005,4.087092
43797,8759,2,-0.558196,-0.231918,-0.134740,-0.039871,0.019372,0.034456,0.047883,0.049972,...,0.101583,0.148900,0.199959,0.319567,0.383164,0.526202,0.536534,0.870811,1.248896,2.527666
43798,8759,3,-0.092768,0.485554,0.714926,0.886660,1.137122,1.305395,1.533551,1.606415,...,2.129681,2.443066,2.557043,2.682588,2.714633,2.897832,3.048069,3.277960,3.351044,3.637214


In [11]:
modelEstimation(quants)

modelEstimation class initialized
Setting up quantiles


AttributeError: 'DataFrame' object has no attribute 'hour'

In [7]:
def cqr_cdf(row, cdf_keyword): 
    #prob = betas.loc[:,('probabilities')].values
    #print(row)
    prob = row.index.values
    quantiles = row.values
    #quantiles[quantiles < 0] = 0
    #quantiles[quantiles > 1] = 1
    quantiles_extended = np.concatenate([[0], sorted(quantiles), [1]])
    probabilities_extended = np.concatenate([[0],prob,[1]])
    if cdf_keyword == 'cdf':
        interpolation = interp1d(quantiles_extended, probabilities_extended, bounds_error=False, fill_value=(0, 1))
    elif cdf_keyword == 'inv_cdf':
        interpolation = interp1d(probabilities_extended, quantiles_extended, bounds_error=False, fill_value=(0, 1))
    return interpolation

class expando:
    pass 

In [8]:
class modelEstimation:
    def __init__(self, qs): 
        print('modelEstimation class initialized')
        print('Setting up quantiles')
        self._set_quantiles(qs)

    def _set_quantiles(self, data): 
        # find quantiles for each hour of the day
        self.betas = expando()
        # slice the data 
        for leadT in data.hour.unique():
            # slice the data
            quantiles = data.loc[data['hour'] == leadT, :].iloc[:, :-1].reset_index(drop=True)
            # set attr
            setattr(self.betas, str(leadT), quantiles)
            del quantiles
            gc.collect()